In [73]:
%matplotlib inline
from glob import glob
import matplotlib as mpl
import os
from joblib import Parallel, delayed
import matplotlib.pyplot as plt


In [74]:
def add_ax_lbls(ax):
    ax.set_xlim(0, 199)
    ax.set_ylim(199, 0)


    xtck_use  = np.arange(-180, 240, 60)
    xtcks_real = np.interp(xtck_use, [-180, 180],  [0, 199])
    ax.set_xticks(xtcks_real)

    ytck_use  = np.arange(0, 240, 60)
    ytcks_real = np.interp(ytck_use, [0, 180],  [0, 199])
    ax.set_yticks(ytcks_real)

    binsx = np.arange(199)
    binsy = np.arange(199)

    YY, XX = np.meshgrid(binsy, binsx)
    
    ax.set_xlabel(r'$\theta_{\mathrm{L}}$ (deg.)')
    tcklbl = ['%i'%tck for tck in xtck_use]
    tcklbl[0] = ''
    tcklbl[2] = ''
    tcklbl[4] = ''
    tcklbl[6] = ''
    ax.set_xticklabels(tcklbl)

    ax.set_ylabel(r'$\phi_{\mathrm{L}}$ (deg.)')
    ax.set_yticklabels(['%i'%tck for tck in ytck_use])
    return ax

In [97]:
%%capture
def create_fig(fl, fig_dir, situation, DD_z_halo, mtype):
    cmap = mpl.cm.viridis
    cmap.set_bad('k')
    print (fl)
    a = np.load(fl, allow_pickle = True)[()]
    aL = a[mtype][situation]['L']
    aM = a[mtype][situation]['M']
    if situation == 'inner':
        vmn = 66
        vmx = 71
    fig, ax = plt.subplots(1,1, facecolor = 'white', figsize = (4,4))
    la = np.log10(aL)
    la[la < vmn] = np.nan
    ax.imshow(la, vmin = vmn, vmax = vmx, cmap = cmap)
    ax = add_ax_lbls(ax)
    
    
    DDname = fl.split('/')[-1].split('_')[-2]
    redshift = float(DD_z_halo[DDname])
    ax.annotate('%s\n%s\nz = %.2f'%(halo, DDname, redshift), (0.05, 0.05), xycoords = 'axes fraction', color = 'white', 
                fontsize = 15, ha = 'left', va = 'bottom')
    ax.annotate('%s'%(mtype.replace('_', ' ')), (0.95, 0.05), xycoords = 'axes fraction', color = 'white', 
                fontsize = 20, ha = 'right', va = 'bottom', fontweight = 'bold')
    
    figname = fig_dir + '/' + fl.split('/')[-1].replace('_rdist.npy', '_%s_%s.png'%(situation, mtype)).replace('Lprof_', '')
    fig.tight_layout()
    fig.savefig(figname, dpi = 500)
    plt.close(fig)

halo, situation, mtype = '5036', 'inner', 'stars'

rdist_dir = '/Users/rsimons/Dropbox/foggie/angular_momentum/profiles/%s/rdist'%halo
mov_dir = '/Users/rsimons/Dropbox/foggie/angular_momentum/figures/thel_phil/mp4s'
DD_z = np.load('/Users/rsimons/Dropbox/foggie/catalogs/DD_redshift.npy', allow_pickle = True)[()]

fig_dir = '/Users/rsimons/Dropbox/foggie/angular_momentum/figures/thel_phil/%s/%s'%(halo, situation)

fls = sort(glob(rdist_dir + '/*npy'))
#for fl in fls: create_fig(fl, fig_dir, situation, DD_z[halo], mtype)
    
Parallel(n_jobs = 2, backend = 'multiprocessing')(delayed(create_fig)(fl, fig_dir, situation, DD_z[halo], mtype) for fl in fls)
mov_file = mov_dir + '/%s_%s_%s.mp4'%(halo, situation, mtype)
os.system("ffmpeg -y -f image2 -r 12 -pattern_type glob -i '%s/*%s_%s.png' "%(fig_dir, situation, mtype) +
          "-vcodec libx264 -crf 25  -pix_fmt yuv420p %s"%mov_file)


KeyboardInterrupt: 

In [91]:
mov_file = mov_dir + '/%s_%s_%s.mp4'%(halo, situation, mtype)
os.system("ffmpeg -y -f image2 -r 12 -pattern_type glob -i '%s/*%s_%s.png' "%(fig_dir, situation, mtype) +
          "-vcodec libx264 -crf 25  -pix_fmt yuv420p %s"%mov_file)

ffmpeg -f -y image2 -r 12 -pattern_type glob -i '/Users/rsimons/Dropbox/foggie/angular_momentum/figures/thel_phil/5036/inner/*inner_stars.png' -vcodec libx264 -crf 25  -pix_fmt yuv420p /Users/rsimons/Dropbox/foggie/angular_momentum/figures/thel_phil/mp4s/5036_inner_stars.mp4


256